In [20]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision import transforms

In [21]:
# Load training data and target and test data
train_data = pd.read_csv("train_data.csv", header=None) / 255.0
train_target = pd.read_csv("train_target.csv", header=None).values.flatten()
test_data = pd.read_csv("test_data.csv", header=None) / 255.0

# Split into training and validation sets
train_data, val_data, train_target, val_labels = train_test_split(
    train_data, train_target, test_size=0.2, random_state=42
)

In [22]:
# process data
class FacialExpressionDataset(Dataset):
    def __init__(self, data, labels=None, transform=None):
        self.data = data.values.astype(np.float32)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Reshape to 48x48
        image = self.data[idx].reshape(48, 48)
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

# Define transformations
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5], std=[0.5])])

In [23]:
# Create datasets
train_dataset = FacialExpressionDataset(pd.DataFrame(train_data), train_target, transform=transform)
val_dataset = FacialExpressionDataset(pd.DataFrame(val_data), val_labels, transform=transform)
test_dataset = FacialExpressionDataset(pd.DataFrame(test_data), transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [25]:
# buidl CNN Model 
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1) 
        self.bn4 = nn.BatchNorm2d(256)
        #self.fc1 = nn.Linear(256, 256)
        self.fc1 = nn.Linear(256 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 256 * 3 * 3)  # Flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Instantiate the model with the updated architecture
num_classes = len(np.unique(train_target))
model = Net(num_classes)

In [26]:
# Define optimizer and loss function
learning_rate = 0.001
loss_fun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.0004)

# Training loop
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = loss_fun(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)

    val_loss = 0.0
    val_acc = 0
    total = 0
    model.eval()
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = loss_fun(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            _, preds = torch.max(outputs, 1)
            val_acc += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = val_acc/total

    # calculate train and validation average and validation accuracy
    avg_train_loss = train_loss/len(train_loader.dataset)
    avg_val_loss = val_loss/len(val_loader.dataset)
    val_accuracy = val_acc/len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

Epoch 1, Train Loss: 1.0077, Val Loss: 0.7861, Val Accuracy: 0.0002
Epoch 2, Train Loss: 0.7704, Val Loss: 0.6950, Val Accuracy: 0.0002
Epoch 3, Train Loss: 0.6768, Val Loss: 0.7193, Val Accuracy: 0.0002
Epoch 4, Train Loss: 0.5969, Val Loss: 0.6348, Val Accuracy: 0.0002
Epoch 5, Train Loss: 0.5649, Val Loss: 0.5805, Val Accuracy: 0.0002
Epoch 6, Train Loss: 0.5128, Val Loss: 0.6108, Val Accuracy: 0.0002
Epoch 7, Train Loss: 0.4724, Val Loss: 0.6227, Val Accuracy: 0.0002
Epoch 8, Train Loss: 0.4270, Val Loss: 0.5732, Val Accuracy: 0.0002
Epoch 9, Train Loss: 0.3835, Val Loss: 0.8143, Val Accuracy: 0.0002
Epoch 10, Train Loss: 0.3301, Val Loss: 0.6471, Val Accuracy: 0.0002
Epoch 11, Train Loss: 0.2767, Val Loss: 0.7835, Val Accuracy: 0.0002
Epoch 12, Train Loss: 0.2475, Val Loss: 0.7799, Val Accuracy: 0.0002
Epoch 13, Train Loss: 0.2026, Val Loss: 0.8127, Val Accuracy: 0.0002
Epoch 14, Train Loss: 0.1923, Val Loss: 0.8811, Val Accuracy: 0.0002
Epoch 15, Train Loss: 0.1582, Val Loss: 1.0

In [27]:
# Predict
predictions = []
model.eval()
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        
# Create submission DataFrame
submission = pd.DataFrame({"Id": range(len(predictions)), "Category": predictions})
submission.to_csv("submission.csv", index=False)